In [1]:
from langchain_ollama import ChatOllama
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
import os
import getpass
from langchain_openai import ChatOpenAI
import requests
from requests.auth import HTTPBasicAuth
import json
import logging
import pandas as pd
from typing import List, Dict, Any, Optional

In [2]:
pd.set_option('display.max_colwidth', None)

In [3]:
# Configure logger
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)  # Set to desired level
handler = logging.StreamHandler()
formatter = logging.Formatter(
    '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
handler.setFormatter(formatter)
logger.addHandler(handler)

In [34]:
USERNAME = "martinbusiness04@gmail.com"
CONFLUENCE_USERNAME = "Akishin@maxim.bayern"

In [5]:
PASSWORD = getpass.getpass("Enter your Atlassian Password: ")

Enter your Atlassian Password:  ········


In [29]:
CONFLUENCE_PASSWORD = getpass.getpass("Enter your Confluence Password: ")

Enter your Confluence Password:  ········


In [11]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

Enter your OpenAI API key:  ········


In [12]:
len(PASSWORD)

192

In [31]:
len(CONFLUENCE_PASSWORD)

192

In [13]:
len(os.environ["OPENAI_API_KEY"])

150

In [14]:
ATLASSIAN_BASE_URL = "https://one-atlas-szdg.atlassian.net"
EPIC_ID = "PLAT-30837"
FIELDS_OF_INTEREST = ["summary", "description", "status", "assignee"]

In [15]:
class JiraRequestor:
    def __init__(self, base_url: str, username: str, password: str):
        """
        Initializes the JiraRequestor with the base URL, username, and password.

        Args:
            base_url (str): The base URL of the Jira instance (e.g., https://your-domain.atlassian.net).
            username (str): The username or email for Jira authentication.
            password (str): The password or API token for Jira authentication.
        """
        self.api_url = f"{base_url}/rest/api/3"
        self.api_search_url = f"{self.api_url}/search"
        self.username = username
        self._password = password

    def _get_all_stories_by_epic_raw(
        self, 
        epic_id: str, 
        fields_of_interest: List[str], 
        max_results: int = 5
    ):
        """
        Fetches all stories associated with a specific epic from Jira.

        Args:
            epic_id (str): The ID or key of the epic.
            fields_of_interest (List[str]): A list of fields to retrieve for each story.
            max_results (int, optional): The number of results to return per request. Defaults to 50.

        Returns:
            Optional[List[Dict[str, Any]]]: A list of stories as dictionaries if successful, else None.
        """
        jql = f'"Epic Link" = "{epic_id}"'
        params = {
            'jql': jql,
            'fields': ','.join(fields_of_interest),
            'maxResults': max_results,
            'startAt': 0
        }

        try:
            logger.debug(f"Fetching stories with params: {params}")
            response = self._make_jira_request(self.api_search_url, params)

            if response is None:
                logger.error("Failed to fetch stories.")
                return None

            logger.info(f"Fetched {len(response.get('issues', []))} stories.")
            return response

        except Exception as e:
            logger.exception(f"An unexpected error occurred: {e}")
            return None

    def get_all_stories_by_epic(
        self, 
        epic_id: str, 
        fields_of_interest: List[str], 
        max_results: int = 5
    ):
        stories_by_epic_data_raw = self._get_all_stories_by_epic_raw(epic_id, fields_of_interest, max_results)
        processed_issues_by_epic = [self._pre_process_issue(story_by_epic_data_raw, FIELDS_OF_INTEREST) for story_by_epic_data_raw in stories_by_epic_data_raw["issues"]]
        df_processed_issues_by_epic = pd.DataFrame(processed_issues_by_epic)
        df_processed_issues_by_epic["issue_type"] = "story"
        return df_processed_issues_by_epic

    def _get_epic_raw(
        self, 
        epic_id: str, 
        fields_of_interest: List[str]
    ) -> Optional[Dict[str, Any]]:
        """
        Fetches details of a specific epic from Jira.

        Args:
            epic_id (str): The ID or key of the epic.
            fields_of_interest (List[str]): A list of fields to retrieve for the epic.

        Returns:
            Optional[Dict[str, Any]]: A dictionary containing epic details if successful, else None.
        """
        issue_url = f"{self.api_url}/issue/{epic_id}"
        params = {
            'fields': ','.join(fields_of_interest)
        }

        try:
            logger.debug(f"Fetching epic with ID: {epic_id} and params: {params}")
            response = self._make_jira_request(issue_url, params)

            if response is None:
                logger.error("Failed to fetch epic.")
                return None

            logger.info(f"Epic {epic_id} fetched successfully.")
            return response

        except Exception as e:
            logger.exception(f"An unexpected error occurred: {e}")
            return None

    def get_epic(self, epic_id: str, fields_of_interest: List[str]):
        epic_data_raw = self._get_epic_raw(epic_id, fields_of_interest)
        pre_processed_epic = self._pre_process_issue(epic_data_raw, fields_of_interest)
        df_pre_processed_epic = pd.DataFrame([pre_processed_epic])
        df_pre_processed_epic["issue_type"] = "epic"
        return df_pre_processed_epic

    def cooked_df_epic_with_stories(self, epic_id, fields_of_interest):
        df_epic = self.get_epic(epic_id, fields_of_interest)
        df_enriched_stories = self.get_all_stories_by_epic(epic_id, fields_of_interest)

        return pd.concat([df_epic, df_enriched_stories], ignore_index=True)

    def format_for_llm(self, cooked_df_epic_with_enriched_stories):
        df = cooked_df_epic_with_enriched_stories
        # Separate epics and stories
        epics = df[df['issue_type'].str.lower() == 'epic']
        stories = df[df['issue_type'].str.lower() == 'story']
    
        formatted_output = []
    
        for _, epic_row in epics.iterrows():
            formatted_output.append("Epic:")
            formatted_output.append(f"title: {epic_row['summary']}")
            formatted_output.append(f"description: {epic_row['description']}")
            formatted_output.append(f"status: {epic_row['status']}")
            formatted_output.append(f"comments_string: {epic_row['comments_string']}")
            formatted_output.append("")
    
        # Since all stories are assumed to be associated with this epic,
        # we'll just include all stories here.
        if not stories.empty:
            formatted_output.append("stories:")
            for _, story_row in stories.iterrows():
                formatted_output.append(f"  title: {story_row['summary']}")
                formatted_output.append(f"  description: {story_row['description']}")
                formatted_output.append(f"  status: {story_row['status']}")
                formatted_output.append(f"  comments_string: {story_row['comments_string']}")
                formatted_output.append("")
    
        return "\n".join(formatted_output).strip()        

    def _pre_process_issue(self, issue, fields_of_interest):
        # Initialize the result dictionary
        result = {}
        result["key"] = issue["key"]
    
        # Helper function to extract text from Jira's "doc" type description
        def extract_description_text(description_field):
            # Check if description is in the doc format
            if isinstance(description_field, dict) and description_field.get('type') == 'doc':
                # Traverse the content array to extract the text
                content = description_field.get('content', [])
                text_parts = []
                for block in content:
                    if block.get('type') == 'paragraph':
                        for inner_content in block.get('content', []):
                            if inner_content.get('type') == 'text':
                                text_parts.append(inner_content.get('text', ''))
                return ' '.join(text_parts).strip()
            # If it's not in doc format (rare cases), just return it as a string
            return str(description_field)
        
        # Extract values based on fields_of_interest
        for field in fields_of_interest:
            if field == "title":
                # title in Jira issue fields is 'summary'
                result[field] = issue.get('fields', {}).get('summary', '')
            elif field == "description":
                # description might be in doc format
                description_field = issue.get('fields', {}).get('description', '')
                result[field] = extract_description_text(description_field)
            elif field == "status":
                # status name is in fields.status.name
                result[field] = issue.get('fields', {}).get('status', {}).get('name', '')
            elif field == "assignee":
                result[field] = issue.get('fields', {}).get('assignee', {}).get('displayName', '')
            else:
                # If any other fields are requested directly from fields
                result[field] = str(issue.get('fields', {}).get(field, ''))

        issue_with_enriched_comments_string = self._enrich_issue_with_comments(result)
        return issue_with_enriched_comments_string

    def _enrich_issue_with_comments(self, issue):
        issue_key = issue["key"]
        df_comments_of_issue = self.get_comments_df_of_issue(issue_key)
        comments_string = self.concatenate_comments(df_comments_of_issue)
        issue["comments_string"] = comments_string 
        return issue

    def _get_comments_of_issue_raw(self, issue_key):
        """
  bb      Get the comments of a specific issue. 

        Args:
            issue_key (str): The key of the issue.
        """
        issue_url = f"{self.api_url}/issue/{issue_key}/comment"
        params = {}

        try:
            logger.debug(f"Fetching story with key: {issue_key} and params: {params}")
            response = self._make_jira_request(issue_url, params)

            if response is None:
                logger.error("Failed to fetch epic.")
                return None

            logger.info(f"Epic {issue_key} fetched successfully.")
            return response

        except Exception as e:
            logger.exception(f"An unexpected error occurred: {e}")
            return None

    def get_comments_df_of_issue(self, issue_key):
        comments_data_raw = self._get_comments_of_issue_raw(issue_key)["comments"]
        pre_processed_comments = []
        for comment_data_raw in comments_data_raw:
            pre_processed_comment = self._pre_process_comment(comment_data_raw) 
            pre_processed_comment["key"] = issue_key
            pre_processed_comments.append(pre_processed_comment)
        
        return pd.DataFrame(pre_processed_comments)

    def concatenate_comments(self, df_comments):
        concatenated_comment_string = []
        # Iterate over rows in the DataFrame
        for idx, row in df_comments.iterrows():
            author = row["author"]
            content = row["content"]
            concatenated_comment_string.append(f"{idx}. {author}: {content};")
        return "\n".join(concatenated_comment_string)
    
    @staticmethod
    def _pre_process_comment(comment):
        def extract_text_from_doc(doc_field):
            # Check if body is in the doc format
            if isinstance(doc_field, dict) and doc_field.get('type') == 'doc':
                content = doc_field.get('content', [])
                text_parts = []
                for block in content:
                    if block.get('type') == 'paragraph':
                        for inner_content in block.get('content', []):
                            if inner_content.get('type') == 'text':
                                text_parts.append(inner_content.get('text', ''))
                return ' '.join(text_parts).strip()
            # If it's not doc format, just return it as string
            return str(doc_field)
        
        author = comment.get('author', {}).get('displayName', '')
        body_field = comment.get('body', {})
        content = extract_text_from_doc(body_field)
    
        return {
            'author': author,
            'content': content
        }
    
    def _make_jira_request(
        self, 
        url: str, 
        params: Dict[str, Any]
    ) -> Optional[Dict[str, Any]]:
        """
        Makes a GET request to the specified Jira API endpoint with given parameters.

        Args:
            url (str): The full URL to send the GET request to.
            params (Dict[str, Any]): Query parameters for the GET request.

        Returns:
            Optional[Dict[str, Any]]: The JSON response as a dictionary if successful, else None.
        """
        try:
            response = requests.get(
                url,
                params=params,
                auth=HTTPBasicAuth(self.username, self._password)
            )

            # Raise an exception for HTTP error responses
            response.raise_for_status()

            data = response.json()
            logger.debug(f"Response data: {json.dumps(data, indent=2)}")
            return data

        except requests.exceptions.HTTPError as http_err:
            logger.error(f"HTTP error occurred: {http_err} - Response: {response.text}")
        except requests.exceptions.RequestException as req_err:
            logger.error(f"Request exception occurred: {req_err}")
        except json.JSONDecodeError as json_err:
            logger.error(f"JSON decode error: {json_err} - Response Text: {response.text}")
        except Exception as err:
            logger.exception(f"An unexpected error occurred: {err}")

        return None

In [16]:
class ConfluenceRequestor:
    def __init__(self, base_url: str, username: str, password: str):
        self.base_url = base_url
        self.username = username
        self._password = password

    def write_to_confluence_page(self, title: str, content: str, space_key: str = "testmax"):
        """Create a new Confluence page with the given title and content."""
        confluence_url = f"{self.base_url}/wiki/rest/api/content"

        # Construct the request body for creating a page
        req_body = {
            "type": "page",
            "title": title,
            "space": {
                "key": space_key
            },
            "body": {
                "storage": {
                    "value": content,
                    "representation": "storage"
                }
            }
        }

        response = self._make_confluence_request(
            url=confluence_url,
            method="POST",
            json_body=req_body
        )
        if response:
            logger.info(f"Page '{title}' created successfully: {json.dumps(response, indent=2)}")
        else:
            logger.error(f"Failed to create page '{title}'.")

    def _make_confluence_request(
        self,
        url: str,
        method: str = "GET",
        params: Optional[Dict[str, Any]] = None,
        json_body: Optional[Dict[str, Any]] = None
    ) -> Optional[Dict[str, Any]]:
        """
        Makes a request to the specified Confluence API endpoint with given parameters or JSON body.

        Args:
            url (str): The full URL to send the request to.
            method (str): The HTTP method to use ('GET' or 'POST', etc.).
            params (Dict[str, Any], optional): Query parameters for the request.
            json_body (Dict[str, Any], optional): JSON body for POST/PUT requests.

        Returns:
            Optional[Dict[str, Any]]: The JSON response as a dictionary if successful, else None.
        """
        try:
            if method.upper() == "GET":
                response = requests.get(
                    url,
                    params=params,
                    auth=HTTPBasicAuth(self.username, self._password)
                )
            elif method.upper() == "POST":
                response = requests.post(
                    url,
                    json=json_body,
                    auth=HTTPBasicAuth(self.username, self._password)
                )
            else:
                logger.error(f"HTTP method '{method}' not supported.")
                return None

            # Raise an exception for HTTP error responses
            response.raise_for_status()

            data = response.json()
            logger.debug(f"Response data: {json.dumps(data, indent=2)}")
            return data

        except requests.exceptions.HTTPError as http_err:
            logger.error(f"HTTP error occurred: {http_err} - Response: {response.text}")
        except requests.exceptions.RequestException as req_err:
            logger.error(f"Request exception occurred: {req_err}")
        except json.JSONDecodeError as json_err:
            logger.error(f"JSON decode error: {json_err} - Response Text: {response.text}")
        except Exception as err:
            logger.exception(f"An unexpected error occurred: {err}")

        return None

In [17]:
# Initialize the JiraRequestor
jira = JiraRequestor(ATLASSIAN_BASE_URL, USERNAME, PASSWORD)

## Get Epic

In [40]:
# jira._get_epic_raw(EPIC_ID, FIELDS_OF_INTEREST)

In [18]:
# df_epic = jira.get_epic(EPIC_ID, FIELDS_OF_INTEREST)
# df_epic

In [21]:
# df_epic["comments_string"]

## Test for Story key PLAT-30837

In [14]:
# jira._get_comments_of_issue_raw("PLAT-30837")["comments"]

In [22]:
# jira.get_comments_df_of_issue("PLAT-30837")

In [17]:
#  jira._get_all_stories_by_epic_raw(EPIC_ID, FIELDS_OF_INTEREST)["issues"]

## Get All Stories By Epic

In [23]:
# df_all_stories_by_epic = jira.get_all_stories_by_epic(EPIC_ID, FIELDS_OF_INTEREST)
# df_all_stories_by_epic

In [18]:
df_cooked_epic_with_stories = jira.cooked_df_epic_with_stories(EPIC_ID, FIELDS_OF_INTEREST)
df_cooked_epic_with_stories

2024-12-08 09:49:57,556 - __main__ - DEBUG - Fetching epic with ID: PLAT-30837 and params: {'fields': 'summary,description,status,assignee'}
2024-12-08 09:49:58,036 - __main__ - DEBUG - Response data: {
  "expand": "renderedFields,names,schema,operations,editmeta,changelog,versionedRepresentations",
  "id": "60523",
  "self": "https://one-atlas-szdg.atlassian.net/rest/api/3/issue/60523",
  "key": "PLAT-30837",
  "fields": {
    "summary": "User Authentication and Authorization System",
    "description": {
      "type": "doc",
      "version": 1,
      "content": [
        {
          "type": "paragraph",
          "content": [
            {
              "type": "text",
              "text": "Develop a robust and secure user authentication and authorization system that allows users to register, log in, manage their profiles, and access features based on their roles. This system should ensure data security, comply with privacy standards, and integrate seamlessly with other modules of t

,key,summary,description,status,assignee,comments_string,issue_type
0,PLAT-30837,User Authentication and Authorization System,"Develop a robust and secure user authentication and authorization system that allows users to register, log in, manage their profiles, and access features based on their roles. This system should ensure data security, comply with privacy standards, and integrate seamlessly with other modules of the application.",Zu erledigen,Mitch Davis,0. Maxim: Maybe we should schedule a meeting to discuss how this will integrate with other modules like analytics or reporting.;\n1. Maxim: This is a great epic! It’s important to clearly define the security and privacy requirements early on to ensure they are addressed during implementation.;,epic
1,PLAT-30842,Role-Based Access Control (RBAC),"Implement role-based access control to define different user roles (e.g., Admin, Editor, Viewer) and assign permissions accordingly. Ensure that users can access only the features and data that their roles permit, enhancing the application's security and usability.",Zu erledigen,Mitch Davis,"0. Maxim: We could consider evaluating a tool or framework like Keycloak to efficiently implement RBAC.;\n1. Maxim: For testing purposes, we can start with mock roles and simple permissions and expand iteratively.;",story
2,PLAT-30841,Profile Management,"Allow authenticated users to view and update their profile information, including personal details, contact information, and password changes. Ensure that all updates are validated and securely stored.",Zu erledigen,Mitch Davis,0. Maxim: We should require users to input their old password before setting a new one for additional security.;\n1. Maxim: What about optional fields like profile pictures? Should we allow users to upload those too?;,story
3,PLAT-30840,Password Recovery,Create a password recovery mechanism where users can request a password reset link sent to their registered email. Implement security measures to prevent unauthorized password resets and ensure the process is user-friendly.,Zu erledigen,Mitch Davis,0. Maxim: Let’s ensure the password reset links have an expiration time to enhance security.;\n1. Maxim: Adding a CAPTCHA during the password reset request could help prevent abuse.;,story
4,PLAT-30839,User Login and Logout,Develop the login functionality enabling users to authenticate using their registered email and password. Ensure secure session management to maintain user sessions and provide a logout option that safely terminates the session.,Zu erledigen,Mitch Davis,"0. Maxim: Should we consider adding social login options (e.g., Google, Facebook)? It could improve user convenience.;\n1. Maxim: We should define password security rules (e.g., minimum length, special characters) to avoid future vulnerabilities.;",story
5,PLAT-30838,User Registration,"Implement a user registration feature that allows new users to create an account by providing necessary information such as username, email, and password. Include validation to ensure data integrity and an email verification process to confirm the user's identity.",Zu erledigen,Mitch Davis,0. Maxim: Don’t forget to implement a timeout for inactive sessions to enhance security.;\n1. Maxim: Should we evaluate JWT vs server-side sessions for session management? It could impact scalability.;,story


In [19]:
question = jira.format_for_llm(df_cooked_epic_with_stories)
question

"Epic:\ntitle: User Authentication and Authorization System\ndescription: Develop a robust and secure user authentication and authorization system that allows users to register, log in, manage their profiles, and access features based on their roles. This system should ensure data security, comply with privacy standards, and integrate seamlessly with other modules of the application.\nstatus: Zu erledigen\ncomments_string: 0. Maxim: Maybe we should schedule a meeting to discuss how this will integrate with other modules like analytics or reporting.;\n1. Maxim: This is a great epic! It’s important to clearly define the security and privacy requirements early on to ensure they are addressed during implementation.;\n\nstories:\n  title: Role-Based Access Control (RBAC)\n  description: Implement role-based access control to define different user roles (e.g., Admin, Editor, Viewer) and assign permissions accordingly. Ensure that users can access only the features and data that their roles p

## Instantiate ChatModel 

In [20]:
DEFAULT_LLAMA_JIRA_SUMMARIZER_PROMPT = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(
        """You are a Project Manager specialized in creating high-level summaries for management. Your summaries should provide an executive overview of the current state of high-level features within a specific epic, focusing on progress, key accomplishments, and any significant issues or dependencies."""
    ),
    HumanMessagePromptTemplate.from_template(
        """{question}

Generate a concise, high-level summary in HTML format suitable for management consumption and compatible with Confluence integration via API. The summary should:

- Use valid HTML tags (e.g., <h2>, <p>, <ul>, <li>, <table>, <tr>, <th>, <td>).
- Include the following sections:
  - <h2>Epic Summary</h2>: A brief description of the epic.
  - <h2>Current Status</h2>: Overall progress (e.g., percentage completed, milestones achieved).
  - <h2>Key Features</h2>: A table listing major features with columns for "Feature", "Status", and "Assignee".
  - <h2>Accomplishments</h2>: Notable achievements since the last update.
  - <h2>Challenges</h2>: Any significant issues or blockers.
  - <h2>Dependencies</h2>: Critical dependencies that may impact progress.
  - <h2>Next Steps</h2>: Upcoming actions or milestones.

Ensure the summary:
- Is clear, concise, and free of technical jargon.
- Provides a high-level perspective understandable to non-technical stakeholders.
- Contains only valid HTML elements.
- The final answer should only consist of the HTML code itself, without any Markdown formatting or code fences.
- Use inner quotes ('') are used for quoting.
""")
])


In [17]:
# # Instantiate the ChatOllama model
# chat_model = ChatOllama(model="llama3.1:8b")

In [21]:
chat_model = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # api_key="...",  # if you prefer to pass api key in directly instaed of using env vars
    # base_url="...",
    # organization="...",
    # other params...
)

## Summarization Step

In [26]:
# Invoke the model with the formatted prompt
summary = chat_model.invoke(DEFAULT_LLAMA_JIRA_SUMMARIZER_PROMPT.format(question=question))
# summary

AIMessage(content='<h2>Epic Summary</h2>\n<p>The User Authentication and Authorization System aims to develop a robust and secure system for user registration, login, profile management, and role-based access. It ensures data security, compliance with privacy standards, and seamless integration with other application modules.</p>\n\n<h2>Current Status</h2>\n<p>Status: Zu erledigen</p>\n<p>Overall progress: 0% completed. Initial planning and requirement definition are underway.</p>\n\n<h2>Key Features</h2>\n<table>\n  <tr>\n    <th>Feature</th>\n    <th>Status</th>\n    <th>Assignee</th>\n  </tr>\n  <tr>\n    <td>Role-Based Access Control (RBAC)</td>\n    <td>Zu erledigen</td>\n    <td>Maxim</td>\n  </tr>\n  <tr>\n    <td>Profile Management</td>\n    <td>Zu erledigen</td>\n    <td>Maxim</td>\n  </tr>\n  <tr>\n    <td>Password Recovery</td>\n    <td>Zu erledigen</td>\n    <td>Maxim</td>\n  </tr>\n  <tr>\n    <td>User Login and Logout</td>\n    <td>Zu erledigen</td>\n    <td>Maxim</td>\n 

In [27]:
summary_content = summary.content
summary_content

'<h2>Epic Summary</h2>\n<p>The User Authentication and Authorization System aims to develop a robust and secure system for user registration, login, profile management, and role-based access. It ensures data security, compliance with privacy standards, and seamless integration with other application modules.</p>\n\n<h2>Current Status</h2>\n<p>Status: Zu erledigen</p>\n<p>Overall progress: 0% completed. Initial planning and requirement definition are underway.</p>\n\n<h2>Key Features</h2>\n<table>\n  <tr>\n    <th>Feature</th>\n    <th>Status</th>\n    <th>Assignee</th>\n  </tr>\n  <tr>\n    <td>Role-Based Access Control (RBAC)</td>\n    <td>Zu erledigen</td>\n    <td>Maxim</td>\n  </tr>\n  <tr>\n    <td>Profile Management</td>\n    <td>Zu erledigen</td>\n    <td>Maxim</td>\n  </tr>\n  <tr>\n    <td>Password Recovery</td>\n    <td>Zu erledigen</td>\n    <td>Maxim</td>\n  </tr>\n  <tr>\n    <td>User Login and Logout</td>\n    <td>Zu erledigen</td>\n    <td>Maxim</td>\n  </tr>\n  <tr>\n  

## Instantiate Confluence Requestor

In [37]:
confluence = ConfluenceRequestor(ATLASSIAN_BASE_URL, CONFLUENCE_USERNAME, CONFLUENCE_PASSWORD)

In [38]:
confluence.write_to_confluence_page("Sprint Documentation 1", summary_content)

2024-12-08 09:54:23,462 - __main__ - DEBUG - Response data: {
  "id": "4227074",
  "type": "page",
  "status": "current",
  "title": "Sprint Documentation 1",
  "space": {
    "id": 3932164,
    "key": "testmax",
    "name": "test-max",
    "type": "collaboration",
    "status": "current",
    "_expandable": {
      "settings": "/rest/api/space/testmax/settings",
      "metadata": "",
      "operations": "",
      "lookAndFeel": "/rest/api/settings/lookandfeel?spaceKey=testmax",
      "identifiers": "",
      "permissions": "",
      "roles": "",
      "icon": "",
      "description": "",
      "theme": "/rest/api/space/testmax/theme",
      "history": "",
      "homepage": "/rest/api/content/3932410"
    },
    "_links": {
      "webui": "/spaces/testmax",
      "self": "https://one-atlas-szdg.atlassian.net/wiki/rest/api/space/testmax"
    }
  },
  "history": {
    "latest": true,
    "createdBy": {
      "type": "known",
      "accountId": "712020:93d347e8-805a-421e-ab5c-5a69a4735785